In [1]:
require 'nn'
require 'hdf5'
require 'optim'


In [3]:
myFile = hdf5.open('../../6-grams.hdf5','r')
data = myFile:all()
myFile:close()
print(data)

{
  test : LongTensor - size: 3761x55
  nwords : LongTensor - size: 1
  train_1000_nocounts : LongTensor - size: 696825x7
  train_nocounts : DoubleTensor - size: 887522x6
  train_1000 : DoubleTensor - size: 887522x6
  train : LongTensor - size: 772670x7
  valid : LongTensor - size: 3370x55
  valid_txt : DoubleTensor - size: 70391x6
  valid_output : LongTensor - size: 3370x50
}


In [4]:
train = data['train_nocounts']:narrow(2,1,6)
train_input = train:narrow(2,1,5)
train_output = train:narrow(2,6,1)

valid_txt = data['valid_txt']:narrow(2,1,6)
valid_txt_input = valid_txt:narrow(2,1,5)
valid_txt_output = valid_txt:narrow(2,6,1)

valid_topredict = data['valid']:narrow(2,1,50)
valid_input = data['valid']:narrow(2,51,5)
valid_output = data['valid_output']

In [4]:
-- Model
nwords = 10001
N = 5
dwin = N
hid1 = 30
hid2 = 100

-- To store the whole model
nnlm = nn.Sequential()

-- Layer to embedd (and put the words along the window into one vector)
LT = nn.Sequential()
LT_ = nn.LookupTable(nwords,hid1)
LT:add(LT_)
LT:add(nn.View(-1, hid1*dwin))

nnlm:add(LT)

concat = nn.ConcatTable()

lin_tanh = nn.Sequential()
lin_tanh:add(nn.Linear(hid1*dwin,hid2))
lin_tanh:add(nn.Tanh())

id = nn.Identity()

concat:add(lin_tanh)
concat:add(id)

nnlm:add(concat)
nnlm:add(nn.JoinTable(2))
nnlm:add(nn.Linear(hid1*dwin + hid2, nwords))
nnlm:add(nn.LogSoftMax())

-- Loss
criterion = nn.ClassNLLCriterion()


nEpochs = 1
batchSize = 32
eta = 0.01
av_L = 0

inputs_batch = torch.DoubleTensor(batchSize,dwin)
targets_batch = torch.DoubleTensor(batchSize)
outputs = torch.DoubleTensor(batchSize, nwords)
dL_do = torch.DoubleTensor(batchSize, nwords)

kag_pred_valid = torch.Tensor(valid_input:size(1),50)
norm_mat = torch.Tensor(valid_input:size(1),50)

In [5]:
parameters, gradParameters = nnlm:getParameters()

In [7]:
for i = 1, 1 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm:updateParameters(eta)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    
    print('Perplexity on valid.txt: '..math.exp(criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze())))
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
    
    
end

Epoch 1: 817.29979610443	
Average Perplexity on train: 1038.2426267938	


/Users/virgileaudi/torch/install/share/lua/5.1/nn/THNN.lua:1091: multi-target not supported at /tmp/luarocks_nn-scm-1-9848/nn/lib/THNN/generic/ClassNLLCriterion.c:18
stack traceback:
	[C]: in function 'v'
	/Users/virgileaudi/torch/install/share/lua/5.1/nn/THNN.lua:1091: in function 'ClassNLLCriterion_updateOutput'
	...udi/torch/install/share/lua/5.1/nn/ClassNLLCriterion.lua:41: in function 'forward'
	[string "for i = 1, 1 do..."]:41: in main chunk
	[C]: in function 'xpcall'
	.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:179: in function <.../virgileaudi/torch/install/share/lua/5.1/itorch/main.lua:143>
	.../virgileaudi/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...rgileaudi/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	.../virgileaudi/torch/install/sh

In [16]:
print (criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze()))

6.4636394432108	70391	


In [28]:
nnlm:forward(valid_input[1]):size()

     1
 10001
[torch.LongStorage of size 2]



In [30]:
kag_pred_valid:zero()
for ii = 1, valid_input:size(1) do
    kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
end

norm_mat:zero()
norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
kag_pred_valid:cdiv(norm_mat)

CE = 0
for iii = 1, valid_input:size(1) do
    mm,aa = valid_output[iii]:max(1)
    CE = CE + math.log(kag_pred_valid[iii][aa[1]])
end

val_res = math.exp(-CE/kag_pred_valid:size(1))
print('Perplexity on valid: '..val_res)



Perplexity on valid: 8.0082783571626	


In [31]:
for i = 1, 1 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm:updateParameters(eta)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    
    print('Perplexity on valid.txt: '..math.exp(criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze())))
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

Epoch 1: 837.54529905319	
Average Perplexity on train: 601.86348771938	


Perplexity on valid.txt: 544.09100456299	


Perplexity on valid: 7.5411670975744	


In [32]:
for i = 3, 8 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm:updateParameters(eta)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    
    print('Perplexity on valid.txt: '..math.exp(criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze())))
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

Epoch 3: 825.9562330246	
Average Perplexity on train: 527.4359496501	


Perplexity on valid.txt: 490.41028578536	


Perplexity on valid: 7.3124630553358	


Epoch 4: 810.81244516373	
Average Perplexity on train: 478.24240750295	


Perplexity on valid.txt: 451.28884307244	


...

Perplexity on valid.txt: 377.90359626936	


Perplexity on valid: 6.7242994567635	


Epoch 8: 843.48118114471	
Average Perplexity on train: 368.57023037956	


Perplexity on valid.txt: 361.76307273096	


Perplexity on valid: 6.6223244922863	


In [33]:
for i = 9, 15 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm:updateParameters(eta)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    
    print('Perplexity on valid.txt: '..math.exp(criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze())))
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

Epoch 9: 827.41940593719	
Average Perplexity on train: 351.92530292078	


Perplexity on valid.txt: 348.08247912807	


Perplexity on valid: 6.5329550483487	


Epoch 10: 818.73921298981	
Average Perplexity on train: 337.5367575987	


Perplexity on valid.txt: 336.26468217226	


...

Perplexity on valid.txt: 301.02181851019	


Perplexity on valid: 6.1939794227997	


Epoch 15: 947.89197707176	
Average Perplexity on train: 285.66596548785	


Perplexity on valid.txt: 294.25169224865	


Perplexity on valid: 6.1404268036459	


In [34]:
for i = 16, 20 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm:updateParameters(0.001)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    
    print('Perplexity on valid.txt: '..math.exp(criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze())))
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

Epoch 16: 884.64695596695	
Average Perplexity on train: 289.63091355636	


Perplexity on valid.txt: 298.53214613822	


Perplexity on valid: 6.161870172494	


Epoch 17: 862.36055397987	
Average Perplexity on train: 287.61173679513	


Perplexity on valid.txt: 298.16840792915	


...

Perplexity on valid.txt: 297.05836750921	


Perplexity on valid: 6.1512939860661	


Epoch 20: 945.45608305931	
Average Perplexity on train: 284.5339143652	


Perplexity on valid.txt: 296.44259460417	


Perplexity on valid: 6.1474060630745	


In [35]:
for i = 21, 22 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm:updateParameters(eta)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    
    print('Perplexity on valid.txt: '..math.exp(criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze())))
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

Epoch 21: 956.9749341011	
Average Perplexity on train: 274.63307799968	


Perplexity on valid.txt: 285.71248287443	


Perplexity on valid: 6.0745420849267	


Epoch 22: 884.85721302032	
Average Perplexity on train: 267.31538454884	


Perplexity on valid.txt: 279.97206629824	


Perplexity on valid: 6.026810149315	


In [36]:
for i = 23, 25 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm:updateParameters(eta)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    
    print('Perplexity on valid.txt: '..math.exp(criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze())))
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

Epoch 23: 925.0414981842	


Average Perplexity on train: 260.75603062799	


Perplexity on valid.txt: 274.84122276941	


Perplexity on valid: 5.9832071111076	


Epoch 24: 861.51717591286	
Average Perplexity on train: 254.65781156113	


Perplexity on valid.txt: 270.11753172733	


Perplexity on valid: 5.942581446567	


Epoch 25: 886.49924206734	
Average Perplexity on train: 248.95055580556	


Perplexity on valid.txt: 265.73155316471	


Perplexity on valid: 5.9045085155161	


In [37]:
for i = 26, 27 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm:updateParameters(eta)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    
    print('Perplexity on valid.txt: '..math.exp(criterion:forward(nnlm:forward(valid_txt_input),valid_txt_output:squeeze())))
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid[ii]:copy(nnlm:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat:zero()
    norm_mat:copy(torch.expandAs(kag_pred_valid:sum(2), kag_pred_valid))
    kag_pred_valid:cdiv(norm_mat)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid:size(1))
    print('Perplexity on valid: '..val_res)
    
end

Epoch 26: 856.12632894516	
Average Perplexity on train: 243.58987799773	


Perplexity on valid.txt: 261.63750741407	


Perplexity on valid: 5.8686917859439	


Epoch 27: 915.89268708229	
Average Perplexity on train: 238.54260990519	


Perplexity on valid.txt: 257.80547119188	


Perplexity on valid: 5.834951373464	


In [39]:
torch.save('../../../nnlm',nnlm)

### Results on test:

In [40]:
test_topredict = data['test']:narrow(2,1,50)
test_input = data['test']:narrow(2,51,5)
kag_pred_test = torch.Tensor(test_input:size(1),50)
norm_mat = torch.Tensor(test_input:size(1),50)

kag_pred_test:zero()
for ii = 1, test_input:size(1) do
    kag_pred_test[ii]:copy(nnlm:forward(test_input[ii]):index(2, test_topredict[ii])):exp()
end

norm_mat:zero()
norm_mat:copy(torch.expandAs(kag_pred_test:sum(2), kag_pred_test))
kag_pred_test:cdiv(norm_mat)

In [43]:
kag_pred_test:size()

 3761
   50
[torch.LongStorage of size 2]



In [44]:
filename = 'bengio_1.f5'
myFile = hdf5.open(filename, 'w')
myFile:write('test', kag_pred_test)
myFile:write('valid',kag_pred_valid)
myFile:close()

### Bengio (no skip):

In [5]:
-- Model
nwords = 10001
N = 5
dwin = N
hid1 = 30
hid2 = 100

-- To store the whole model
nnlm2 = nn.Sequential()

-- Layer to embedd (and put the words along the window into one vector)
LT2 = nn.Sequential()
LT2_ = nn.LookupTable(nwords,hid1)
LT2:add(LT2_)
LT2:add(nn.View(-1, hid1*dwin))

nnlm2:add(LT2)

lin_tanh2 = nn.Sequential()
lin_tanh2:add(nn.Linear(hid1*dwin,hid2))
lin_tanh2:add(nn.Tanh())

nnlm2:add(lin_tanh2)
nnlm2:add(nn.Linear(hid2, nwords))
nnlm2:add(nn.LogSoftMax())

-- Loss
criterion2 = nn.ClassNLLCriterion()

batchSize = 32
eta = 0.01
av_L = 0

inputs_batch = torch.DoubleTensor(batchSize,dwin)
targets_batch = torch.DoubleTensor(batchSize)
outputs = torch.DoubleTensor(batchSize, nwords)
dL_do = torch.DoubleTensor(batchSize, nwords)

kag_pred_valid2 = torch.Tensor(valid_input:size(1),50)
norm_mat2 = torch.Tensor(valid_input:size(1),50)

valtxtperp = torch.Tensor(25)
valkagperp = torch.Tensor(25)

In [6]:
for i = 1, 25 do
    -- timing the epoch
    timer = torch.Timer()
    av_L = 0
    
    -- max renorm
    LT2_.weight:renorm(2,1,1)
    
    -- mini batch loop
    for t = 1, train_input:size(1), batchSize do
        -- Mini batch data
        current_batch_size = math.min(batchSize,train_input:size(1)-t)
        inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
        targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
        
        -- reset gradients
        nnlm2:zeroGradParameters()
        --gradParameters:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs:narrow(1,1,current_batch_size):copy(nnlm2:forward(inputs_batch:narrow(1,1,current_batch_size)))

        -- Average loss computation
        L = criterion2:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
        av_L = av_L + L

        -- Backward pass
        dL_do:narrow(1,1,current_batch_size):copy(criterion2:backward(outputs:narrow(1,1,current_batch_size),
                targets_batch:narrow(1,1,current_batch_size)))
        nnlm2:backward(inputs_batch:narrow(1,1,current_batch_size), dL_do:narrow(1,1,current_batch_size))
        nnlm2:updateParameters(eta)
        
    end
        
    print('Epoch '..i..': '..timer:time().real)
    print('Average Perplexity on train: '.. math.exp(av_L/math.floor(train_input:size(1)/batchSize)))
    
    
    -- Evaluating perplexity on valiadation_txt:
    perp_txt = math.exp(criterion2:forward(nnlm2:forward(valid_txt_input),valid_txt_output:squeeze()))
    valtxtperp[i] = perp_txt
    print('Perplexity on valid.txt: '..perp_txt)
    
    -- Evaluatin perplexity on validation kaggle:
    kag_pred_valid2:zero()

    for ii = 1, valid_input:size(1) do
        kag_pred_valid2[ii]:copy(nnlm2:forward(valid_input[ii]):index(2, valid_topredict[ii])):exp()
    end
    
    norm_mat2:zero()
    norm_mat2:copy(torch.expandAs(kag_pred_valid2:sum(2), kag_pred_valid2))
    kag_pred_valid2:cdiv(norm_mat2)
    
    CE = 0
    for iii = 1, valid_input:size(1) do
        mm,aa = valid_output[iii]:max(1)
        CE = CE + math.log(kag_pred_valid2[iii][aa[1]])
    end
    
    val_res = math.exp(-CE/kag_pred_valid2:size(1))
    valkagperp[i] = val_res
    print('Perplexity on valid: '..val_res)
    
end

Epoch 1: 440.03609108925	
Average Perplexity on train: 1082.4594136257	


Perplexity on valid.txt: 680.00174311834	


Perplexity on valid: 8.0683115316289	


Epoch 2: 437.69223618507	
Average Perplexity on train: 625.75434648624	


Perplexity on valid.txt: 565.59387298635	


...

Perplexity on valid.txt: 258.80916918393	


Perplexity on valid: 5.7126058674399	


Epoch 25: 437.70601701736	
Average Perplexity on train: 235.57116926444	


Perplexity on valid.txt: 255.38467631097	


Perplexity on valid: 5.6819592787889	


In [8]:
test_topredict = data['test']:narrow(2,1,50)
test_input = data['test']:narrow(2,51,5)
kag_pred_test2 = torch.Tensor(test_input:size(1),50)
norm_mat2 = torch.Tensor(test_input:size(1),50)

kag_pred_test2:zero()
for ii = 1, test_input:size(1) do
    kag_pred_test2[ii]:copy(nnlm2:forward(test_input[ii]):index(2, test_topredict[ii])):exp()
end

norm_mat2:copy(torch.expandAs(kag_pred_test2:sum(2), kag_pred_test2))
kag_pred_test2:cdiv(norm_mat2)

In [9]:
filename = 'bengio_2.f5'
myFile = hdf5.open(filename, 'w')
myFile:write('test', kag_pred_test2)
myFile:write('valid',kag_pred_valid2)
myFile:close()

In [10]:
filename = 'bengio_2_perp.f5'
myFile = hdf5.open(filename, 'w')
myFile:write('test', valtxtperp)
myFile:write('valid',valkagperp)
myFile:close()

In [11]:
torch.save('../../../nnlm2',nnlm2)